# Estimación de dirección de arribo de fuentes sonoras 

In [1]:
import numpy as np
import pandas as pd
import gspread # Extension para leer el excel desde Drive
from oauth2client.service_account import ServiceAccountCredentials # Extensión para autentificación con Drive
from tdoa_py import estimation, simulation_room

In [2]:
# Configurar credenciales
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("simulationsdoa-credenciales.json", scope)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/13XTDng98P99pfexK78Dd4Gud1CzZwO7PfVhpyIG1jCM/edit?gid=0#gid=0")
sheet = spreadsheet.sheet1

# Leer datos
data = sheet.get_all_records()
df_simulations = pd.DataFrame(data)

df_simulations["room_dim"] = df_simulations.apply(lambda row: (row["room_x"], row["room_y"], row["room_z"]), axis=1) # Para pasar el recinto en una tupla luego
df_simulations.head() # Verificar el resultado

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,theta_2,theta_3,t_12,t_13,t_14,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0.00,0.00,0.000292,0.000583,0.000875,0.00,0.000583,,,"(100, 100, 100)"
1,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0.00,0.00,0.000292,0.000583,0.000875,0.00,0.000583,,,"(100, 100, 100)"
2,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0.00,12.53,0.000292,0.000583,0.000854,4.18,0.000576,,,"(100, 100, 100)"
3,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15.27,12.42,0.000271,0.000563,0.000854,16.47,0.000563,,,"(100, 100, 100)"
4,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,21.73,21.73,0.000271,0.000542,0.000813,21.73,0.000542,,,"(100, 100, 100)"


In [3]:
# Iterar sobre el DataFrame con indexación correcta
for idx, sim in df_simulations.iterrows():
    src_az_deg, mic_signals = simulation_room.sim_room_Nmics(
        wav_path=f'audios/anechoic/{sim["audio"]}',
        out_dir=f'audios/output/{sim["sim_name"]}',
        fs=48000,
        room_dim=sim["room_dim"],
        rt60=sim["rt60"],
        snr_db=sim["snr_db"],
        n_mics=sim["n_mics"],
        mic_d=sim["mic_d"],
        mic_z=sim["mic_z"],
        mic_directivity=sim["mic_directivity"],
        src_dist=sim["src_dist"],
        src_az_deg=sim["src_az_deg"],
        src_z=sim["src_z"],
        save_audio=False
    )
    
    avg_angle, avg_tdoa, angles, tdoas = estimation.estimate_doa(
        signals=mic_signals,
        fs=48000,
        d=sim["mic_d"],
        method=sim["method"]
    )

    # Guardar resultados en cada fila individual
    # print(angles)
    # print(tdoas)
    df_simulations.at[idx, 'avg_angle_deg'] = avg_angle
    df_simulations.at[idx, 'avg_tdoa'] = avg_tdoa
    df_simulations.at[idx, 'theta_1'] = angles[0]
    df_simulations.at[idx, 'theta_2'] = angles[1]
    df_simulations.at[idx, 'theta_3'] = angles[2]
    df_simulations.at[idx, 't_12'] = tdoas[0]
    df_simulations.at[idx, 't_13'] = tdoas[1]
    df_simulations.at[idx, 't_14'] = tdoas[2]
    # print("phi_deg", phi_deg)
    # print("tdoas", tdoas)

# # Convertir el DataFrame a una lista de listas para Google Sheets
df_simulations["room_dim"] = df_simulations["room_dim"].apply(lambda x: ",".join(map(str, x)))
data = [df_simulations.columns.values.tolist()] + df_simulations.values.tolist()

# Escribir en la hoja de cálculo
sheet.update(data)
print("DataFrame guardado correctamente en Google Sheets.")
df_simulations



[0.1, 0.2, 0.30000000000000004]
[np.float64(0.0002708333333333333), np.float64(0.0005625), np.float64(0.0008541666666666667)]
args [np.float64(0.9289583333333332), np.float64(0.9646874999999999), np.float64(0.9765972222222221)]
max_val 0.9765972222222221
norm_args [np.float64(0.9289583333333332), np.float64(0.9646874999999999), np.float64(0.9765972222222221)]
[0.1, 0.2, 0.30000000000000004]
[np.float64(0.0002708333333333333), np.float64(0.0005625), np.float64(0.0008541666666666667)]
args [np.float64(0.9289583333333332), np.float64(0.9646874999999999), np.float64(0.9765972222222221)]
max_val 0.9765972222222221
norm_args [np.float64(0.9289583333333332), np.float64(0.9646874999999999), np.float64(0.9765972222222221)]
[0.1, 0.2, 0.30000000000000004]
[np.float64(0.0002708333333333333), np.float64(0.0006458333333333333), np.float64(0.0008333333333333334)]
args [np.float64(0.9289583333333332), np.float64(1.1076041666666665), np.float64(0.9527777777777776)]
max_val 1.1076041666666665
norm_args

,sim_name,method,audio,room_x,room_y,room_z,rt60,snr_db,n_mics,mic_d,...,theta_2,theta_3,t_12,t_13,t_14,avg_angle_deg,avg_tdoa,hemi_1,hemi_2,room_dim
0,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15.271729,12.420011,0.000271,0.000562,0.000854,16.472908,0.000562,,,"100,100,100"
1,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,15.271729,12.420011,0.000271,0.000562,0.000854,16.472908,0.000562,,,"100,100,100"
2,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0.000000,30.659262,0.000271,0.000646,0.000833,21.218383,0.000583,,,"100,100,100"
3,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,21.726984,33.521302,0.000208,0.000542,0.000729,33.213075,0.000493,,,"100,100,100"
4,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,0.000000,17.146210,0.000250,0.000625,0.000896,18.005369,0.000590,,,"100,100,100"
5,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,38.182877,28.198061,0.000250,0.000458,0.000771,32.447969,0.000493,,,"100,100,100"
6,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,34.737544,38.182877,0.000229,0.000479,0.000687,37.034433,0.000465,,,"100,100,100"
7,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,38.182877,42.404466,0.000167,0.000458,0.000646,45.240271,0.000424,,,"100,100,100"
8,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,55.133471,42.404466,0.000208,0.000333,0.000646,47.309625,0.000396,,,"100,100,100"
9,1,roth,audio_anecoico_corto.wav,100,100,100,3,70,4,0.1,...,47.245832,46.309392,0.000188,0.000396,0.000604,47.843328,0.000396,,,"100,100,100"
